# Adopt Timeliness

## Contents
#### Setup
1. [Import packages and options](#import_packages)
2. [Define key variables](#define_key_variables)

3. [Adopt_applications_data_sorted](#Adopt_applications_data_sorted) - Orders adopt_application_5 by case_number and application date
4. [Adopt_applications_temp](#Adopt_applications_temp) - takes the adopt_applications_data sorted table and filters it so it only has the first application record per case number
5. [adopt_orders_data_sorted](#adopt_orders_data_sorted) - Sorts the disposals table by case_number and receipt date, removing contact orders, placement revoke or vary orders and other type orders
6. [adopt_orders_temp](#adopt_orders_temp) - takes the adopt_orders_data_sorted table and filters it so it only has the first disposal record per case number
7. [adopt_apps_and_orders_match](#adopt_apps_and_orders_match) - calculates the wait_weeks from the disposal date and the application date


8. [Adopt_case_data_temp](#Adopt_case_data_temp) - filters by year>2010 and by first application date
9. [Applicant_Info](#Applicant_Info) - joins the roles, parties and address tables for applicants to get information on the applicants
10. [adopt_respondent_info](#adopt_respondent_info) - joins the roles, parties and address tables for respondents to get information on the respondents
11. [applicants_3](#applicants_3) takes the adopt_applicant_info table and reformats it, decoding gender and representative role values into strings

12. [Adopt_respondents_temp](#Adopt_respondents_temp) - takes the adopt_respondent_info table and reformats it, decoding gender and representative role values into strings
13. [Adopt_app_rep](#Adopt_app_rep) - joins the case data with the applicant representation data
14. [Adopt_resp_rep](#Adopt_resp_rep) - joins the case data with the respondent representation data
15. [Adopt_Hearing_Events](#Adopt_Hearing_Events) - takes the adoption hearing events from the hearings table and joins it with the events table data 
16. [Adopt_Hearings_Cases](#Adopt_Hearings_Cases) - takes the hearing events, filters by adoption event codes and adds a flag whether the hearing is the first in the case
17. [Hearing_Adopt_Applicants](#Hearing_Adopt_Applicants) - joins the cases from applicant representation table to the first hearing for the case in the adopt hearings cases table

18. [Hearing_Adopt_Respondents](#Hearing_Adopt_Respondents) - joins the cases from respondents representation table to the first hearing for the case in the adopt hearings cases table 
19. [Adopt_App](#Adopt_App) - Groups the hearing_adopt_applicants table and produces a count per group
20. [Adopt_Resp](#Adopt_Resp) - Groups the hearing_adopt_respondents table and produces a count per group
21. [adopt_case](#adopt_case) - groups and formats adopt_case_data_v3 table and gives a count for each group
22. [Adopt_Case_Hearings](#Adopt_Case_Hearings) - creates a count of all the cases with a hearing per quarter
23. [Adoption](#Adoption) - Joins the applicant/respondent representation count tables, case count table, and case hearing count tables 
24. [Applicant_representation](#Applicant_representation) - creates a table showing whether all, some, or none of the applicants for a case have representation
25. [Respondent_representation](#Respondent_representation) - creates a table showing whether all, some, or none of the respondents for a case have representation
26. [Adopt_Disposals_Final](#Adopt_Disposals_Final) - joins the adopt_apps_and_orders_match table and the representation tables, joining timeliness and legal representation data for a case together
27. [Adopt_Quarterly](#Adopt_Quarterly) - groups the disposals_final table by quarter, providing a total and mean wait_weeks per quarter
28. [Adopt_Annual](#Adopt_Annual) - groups the disposals_final table annually, providing a total and mean wait_weeks annually 
29. [Adopt_timeliness_combined](#Adopt_timeliness_combined) - combines and orders the annual and quarterly timeliness data


## 1. Import packages and set options 
<a name="import_packages"></a>

In [1]:
import pandas as pd  # a module which provides the data structures and functions to store and manipulate tables in dataframes
import pydbtools as pydb  # A module which allows SQL queries to be run on the Analytical Platform from Python, see https://github.com/moj-analytical-services/pydbtools
import boto3  # allows you to directly create, update, and delete AWS resources from Python scripts

# sets parameters to view dataframes for tables easier
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)


## 2. Define key variables to be used throughout the notebook 
<a name="define_key_variables"></a>

In [2]:
#this is the database we will be extracting from
database = "familyman_dev_v3" 

#this extracts the latest snapshot from athena
get_snapshot_date = f"SELECT mojap_snapshot_date from {database}.events order by mojap_snapshot_date desc limit 1"
snapshot_date = str(pydb.read_sql_query(get_snapshot_date)['mojap_snapshot_date'].values[0])

#this is the athena database we will be storing our tables in
fcsq_database = "fcsq"

#this is the s3 bucket we will be saving data to
s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")

#change these to the current quarter and year not the quarter being published
latest_quarter = 3
latest_year = 2022

In [3]:
"""
CREATE EXTERNAL TABLE IF NOT EXISTS `fcsq`.`COURT_MV_FEB21_DFJ` (
  `CODE` int,
  `COURT` string,
  `NAME` string,
  `Court_Type` string,
  `DFJ_Area` string,
  `Region` string,
  `Court_pre_2014` string,
  `Region_Pre2014` string,
  `DFJ_New` string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe' 
WITH SERDEPROPERTIES (
  'serialization.format' = ',',
  'field.delim' = ','
) LOCATION 's3://alpha-family-data/fcsq_processing/lookups/COURT_MV_FEB21_DFJ/'
TBLPROPERTIES ('has_encrypted_data'='false');
"""

"\nCREATE EXTERNAL TABLE IF NOT EXISTS `fcsq`.`COURT_MV_FEB21_DFJ` (\n  `CODE` int,\n  `COURT` string,\n  `NAME` string,\n  `Court_Type` string,\n  `DFJ_Area` string,\n  `Region` string,\n  `Court_pre_2014` string,\n  `Region_Pre2014` string,\n  `DFJ_New` string\n)\nROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe' \nWITH SERDEPROPERTIES (\n  'serialization.format' = ',',\n  'field.delim' = ','\n) LOCATION 's3://alpha-family-data/fcsq_processing/lookups/COURT_MV_FEB21_DFJ/'\nTBLPROPERTIES ('has_encrypted_data'='false');\n"

## 3. Adopt_applications_data_sorted table - Orders adopt_application_5 by case_number and application date
<a name="Adopt_applications_data_sorted"></a>

### Drop the adopt_applications_data_sorted table if it already exists and remove its data from the S3 bucket

In [4]:
drop_adopt_applications_data_sorted = f"""
DROP TABLE IF EXISTS fcsq.adopt_applications_data_sorted;
"""
pydb.start_query_execution_and_wait(drop_adopt_applications_data_sorted)

# clean up previous adopt_applications_data_sorted files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_applications_data_sorted/").delete();

### Create the adopt_applications_data_sorted table in Athena

In [5]:
create_adopt_applications_data_sorted = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_applications_data_sorted
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_applications_data_sorted') AS
SELECT * FROM
fcsq.adopt_application_5
ORDER BY 
CASE_NUMBER, APP_DATE
"""

pydb.start_query_execution_and_wait(create_adopt_applications_data_sorted);



#### adopt_applications_data_sorted validation

In [6]:
adopt_applications_data_sorted_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_applications_data_sorted")
adopt_applications_data_sorted_count


,count
0,183890


## 4. Create adopt_applications temporary tables - takes the adopt_applications_data sorted table and filters it so it only has the first application record per case number
<a name="Adopt_applications_temp"></a>

In [7]:
create_adopt_applications_1 = f"""
SELECT *, row_number() over (order by CASE_NUMBER, APP_DATE) as SEQ_NUM
FROM fcsq.adopt_applications_data_sorted
"""
pydb.create_temp_table(create_adopt_applications_1,'adopt_applications_1')

create_adopt_applications_2 = f"""
SELECT DISTINCT case_number, app_type, min(seq_num) as min_of_seq_num
FROM __temp__.adopt_applications_1 GROUP BY case_number, app_type
"""
pydb.create_temp_table(create_adopt_applications_2,'adopt_applications_2')

create_adopt_applications_3 = f"""
SELECT
    t1.case_number,
    t2.App_date,
    t2.year,
    t2.quarter,
    t2.court,
    t2.app_type,
    t2.Case_app_type, 
    t2.Adoption, 
    t2.Contested, 
    t2.Standard, 
    t2.Convention, 
    t2.Foreign, 
    t2.Placement, 
    t2.Placement_revoke_or_vary, 
    t2.Contact_s26, 
    t2.Contact_s26_revoke_or_vary, 
    t2.Change_surname, 
    t2.Remove_child_from_UK, 
    t2.Other_order_type, 
    t2.Adoption_Cases, 
    t2.Non_Adoption_Cases
FROM __temp__.ADOPT_APPLICATIONS_2 t1 LEFT JOIN
__temp__.ADOPT_APPLICATIONS_1 t2 ON (t1.MIN_of_seq_num = t2.Seq_Num)
"""
pydb.create_temp_table(create_adopt_applications_3,'adopt_applications_3')

## 5. adopt_orders_data_sorted table - Sorts the disposals table by case_number and receipt date, removing contact orders, placement revoke or vary orders and other type orders
<a name="adopt_orders_data_sorted"></a>

### Drop the adopt_orders_data_sorted table if it already exists and remove its data from the S3 bucket

In [8]:
drop_adopt_orders_data_sorted = f"""
DROP TABLE IF EXISTS fcsq.adopt_orders_data_sorted;
"""
pydb.start_query_execution_and_wait(drop_adopt_orders_data_sorted)

# clean up previous adopt_orders_data_sorted files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_orders_data_sorted/").delete();

### Create the adopt_orders_data_sorted table in Athena

In [9]:
create_adopt_orders_data_sorted = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_orders_data_sorted
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_orders_data_sorted') AS
SELECT * 
FROM fcsq.adopt_disposals5 t1
    WHERE t1.Type in ('Standard','Convention','?Foreign?')
    ORDER BY t1.CASE_NUMBER, t1.Receipt_date;
"""
pydb.start_query_execution_and_wait(create_adopt_orders_data_sorted);

In [10]:
"""

#t1.Type != 'Contact_s26' AND t1.Type != 'Contact_s26_revoke_or_vary' AND t1.Type != 
 'Placement_revoke_or_vary' AND t1.Type != 'Other_order_type'
 
"""

"\n\n#t1.Type != 'Contact_s26' AND t1.Type != 'Contact_s26_revoke_or_vary' AND t1.Type != \n 'Placement_revoke_or_vary' AND t1.Type != 'Other_order_type'\n \n"

#### adopt_orders_data_sorted validation

In [11]:
adopt_orders_data_sorted_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_orders_data_sorted")
adopt_orders_data_sorted_count

,count
0,85548


## 6. Create adopt_orders temporary tables - takes the adopt_orders_data_sorted table and filters it so it only has the first disposal record per case number
<a name="adopt_orders_temp"></a>

In [12]:
create_adopt_orders_0 = f"""
SELECT t1.CASE_NUMBER, 
          t1.Court, 
          t1.Year, 
          t1.Quarter, 
          t2.App_type, 
          t2.App_date, 
          t1.Receipt_date AS Disp_Date, 
          t1.EVENT_MODEL, 
          t1.FIELD_MODEL, 
          t1.Order_type, 
          t1.Country_of_birth, 
          t1.Number_applicants, 
          t1.Adopter_type, 
          t1.Adopter, 
          t1.Child_sex, /*Changed this from child_sex2 as the process has changed, check to see if an error comes up*/ 
          t1.Age_band, 
          t1.Child_age, 
          t1.Adoption, 
          t1.Type, 
          t1.Adopter_2, 
          DAY(t1.Receipt_date -t2.App_date) as DATE_DIFF
      FROM fcsq.ADOPT_ORDERS_DATA_SORTED t1
           INNER JOIN __temp__.ADOPT_APPLICATIONS_3 t2 ON (t1.CASE_NUMBER = t2.CASE_NUMBER)
                 WHERE (DAY(t1.Receipt_date -t2.App_date)) >= 0;
"""
pydb.create_temp_table(create_adopt_orders_0,'adopt_orders_0')


create_adopt_orders_1 = f"""
   SELECT t1.CASE_NUMBER, 
          t1.Court, 
          t1.Year, 
          t1.Quarter, 
          t1.App_type, 
          t1.EVENT_MODEL AS Disp_Type, 
          t1.App_date, 
          t1.Disp_Date, 
          t1.FIELD_MODEL, 
          t1.Order_type, 
          t1.Number_applicants, 
          t1.Child_sex, /*Changed this from child_sex2 as the process has changed, check to see if an error comes up*/ 
          t1.Adoption, 
          t1.Type, 
          row_number() over (order by CASE_NUMBER,App_type,App_date,disp_date) as Seq_no
      FROM __temp__.ADOPT_ORDERS_0 t1
      ORDER BY t1.CASE_NUMBER,
               t1.App_type,
               t1.App_date;
"""

pydb.create_temp_table(create_adopt_orders_1,'adopt_orders_1')

create_adopt_orders_2 = f"""
SELECT DISTINCT CASE_NUMBER, 
          App_type, 
          App_date, 
        (MIN(Seq_No)) AS MIN_of_Seq_No
      FROM __temp__.ADOPT_ORDERS_1 t1
      GROUP BY CASE_NUMBER,
               App_type,
               App_date;
"""

pydb.create_temp_table(create_adopt_orders_2,'adopt_orders_2')

create_adopt_orders_3 = f"""
   SELECT DISTINCT t1.CASE_NUMBER, 
          t1.App_type, 
          t2.Court, 
          t1.App_date, 
          t2.Disp_Date, 
          t2.Disp_Type, 
          t2.Year, 
          t2.Quarter, 
          t2.Order_type, 
          t2.Adoption, 
          t2.Type
      FROM __temp__.ADOPT_ORDERS_2 t1
           LEFT JOIN __temp__.ADOPT_ORDERS_1 t2 ON (t1.MIN_of_Seq_No = t2.Seq_No);
"""
pydb.create_temp_table(create_adopt_orders_3,'adopt_orders_3')


In [13]:
adopt_orders_3_check = pydb.read_sql_query("SELECT count(*) as count from __temp__.adopt_orders_3")
adopt_orders_3_check

,count
0,84872


## 7. adopt_apps_and_orders_match table - calculates the wait_weeks from the disposal date and the application date
<a name="adopt_apps_and_orders_match"></a>

### Drop the adopt_apps_and_orders_match table if it already exists and remove its data from the S3 bucket

In [14]:
drop_adopt_apps_and_orders_match = f"""
DROP TABLE IF EXISTS fcsq.adopt_apps_and_orders_match;
"""
pydb.start_query_execution_and_wait(drop_adopt_apps_and_orders_match)

# clean up previous adopt_apps_and_orders_match files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/ADOPT_APPS_AND_ORDERS_MATCH/").delete();

### Create the adopt_apps_and_orders_match table in Athena

In [15]:
create_adopt_apps_and_orders_match =f"""
CREATE TABLE IF NOT EXISTS fcsq.ADOPT_APPS_AND_ORDERS_MATCH
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/ADOPT_APPS_AND_ORDERS_MATCH') AS
   SELECT t2.CASE_NUMBER, 
          t2.App_type, 
          t2.Court, 
          t2.App_date, 
          t2.Disp_Date, 
          /* Wait_weeks */
            (CAST (DAY(t2.Disp_Date-t2.App_date) as double)/7) AS Wait_weeks, 
          t2.Disp_Type, 
          t2.Year, 
          t2.Quarter, 
          t2.Order_type, 
          t2.Adoption, 
          t2.Type, 
          /* DSP_COURT */
          t2.Court AS DSP_COURT
      FROM __temp__.ADOPT_ORDERS_3 t2;


"""
pydb.start_query_execution_and_wait(create_adopt_apps_and_orders_match);

#### ADOPT_APPS_AND_ORDERS_MATCH validation

In [16]:
adopt_apps_and_orders_match_count = pydb.read_sql_query("select count(*) as count from fcsq.ADOPT_APPS_AND_ORDERS_MATCH")
adopt_apps_and_orders_match_count

,count
0,84872


In [17]:
pydb.read_sql_query("select * from fcsq.adopt_apps_and_orders_match where case_number='LV20Z04103'")

,case_number,app_type,court,app_date,disp_date,wait_weeks,disp_type,year,quarter,order_type,adoption,type,dsp_court
0,LV20Z04103,AO,251,2020-11-25,2021-08-18,38.0,A76,2021,3,<NA>,Adoption,Standard,251


## 8. Adopt_case_data temporary tables - filters by year>2010 and by first application date
<a name="Adopt_case_data_temp"></a>

In [18]:
create_adopt_case_data_v1 = f"""
SELECT T1.YEAR,
            T1.QUARTER,
            T1.COURT,
            T1.CASE_NUMBER,
            T1.APP_TYPE,
            T1.CASE_APP_TYPE,
            T1.ADOPTION,
            T1.HIGH_COURT,
            T1.CONTESTED,
            T1.NUMBER_APPLICANTS,
            T1.ADOPTER_TYPE,
            date_format(T1.APP_DATE,'%d-%m-%Y') AS APP_DATE2
    FROM fcsq.adopt_apps_6_adoptions_only AS t1
    ORDER BY case_number, app_date2, court;
    
"""

create_adopt_case_data_v2 = f"""
SELECT *,(case when row_number() over (partition by case_number order by 
        APP_DATE2) = 1 then 1 else 0 end) as case_number_id
FROM __temp__.adopt_case_data_v1
"""

create_adopt_case_data_v3 = f"""
SELECT *
FROM __temp__.adopt_case_data_v2
where case_number_id = 1 and year > 2010;
"""
pydb.create_temp_table(create_adopt_case_data_v1,'adopt_case_data_v1')

pydb.create_temp_table(create_adopt_case_data_v2,'adopt_case_data_v2')
pydb.create_temp_table(create_adopt_case_data_v3,'adopt_case_data_v3')





In [19]:
adopt_case_data_v3_count = "SELECT count(*) as count from __temp__.adopt_case_data_v3"
pydb.read_sql_query(adopt_case_data_v3_count)

,count
0,62462


## 9. Applicant_Info table - joins the roles, parties and address tables for applicants to get information on the applicants
<a name="Applicant_Info"></a>

### Drop the Applicant_Info table if it already exists and remove its data from the S3 bucket

In [20]:
drop_Adopt_Applicant_Info = f"""
DROP TABLE IF EXISTS fcsq.Adopt_Applicant_Info;
"""
pydb.start_query_execution_and_wait(drop_Adopt_Applicant_Info)

# clean up previous Adopt_Applicant_Info files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_Applicant_Info/").delete();

### Create the Applicant_Info table in Athena

In [21]:
create_Adopt_Applicant_Info = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_Applicant_Info
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_Applicant_Info') AS
 SELECT DISTINCT
   {database}.roles.ROLE, 
   {database}.roles.REPRESENTATIVE_ROLE, 
   {database}.roles.ROLE_MODEL, 
   {database}.roles.PARTY, 
   {database}.roles.CASE_NUMBER, 
   {database}.parties.PERSON_GIVEN_FIRST_NAME, 
   {database}.parties.PERSON_FAMILY_NAME, 
   {database}.parties.COMPANY, 
   {database}.addresses.POSTCODE, 
   {database}.parties.GENDER, 
   {database}.roles.DELETE_FLAG
FROM 
  ({database}.roles INNER JOIN {database}.parties ON {database}.roles.PARTY = {database}.parties.PARTY) 
  INNER JOIN {database}.addresses ON {database}.roles.ADDRESS = {database}.addresses.ADDRESS
WHERE 
    (((({database}.roles.ROLE_MODEL)= 'APLZ') AND (({database}.roles.DELETE_FLAG)= 'N')) 
    OR ((({database}.roles.ROLE_MODEL)= 'APLA') AND (({database}.roles.DELETE_FLAG)= 'N')))
    AND {database}.roles.mojap_snapshot_date = date '{snapshot_date}'
    AND {database}.parties.mojap_snapshot_date = date '{snapshot_date}'
    AND {database}.addresses.mojap_snapshot_date = date '{snapshot_date}';
"""

pydb.start_query_execution_and_wait(create_Adopt_Applicant_Info);



In [22]:
pydb.read_sql_query("SELECT * from fcsq.adopt_applicant_info where case_number='NE19Z02909'")

,role,representative_role,role_model,party,case_number,person_given_first_name,person_family_name,company,postcode,gender,delete_flag
0,10712248,<NA>,APLZ,8748181,NE19Z02909,Christopher Martyn,Urwin,<NA>,<NA>,1,N


#### Applicant_Info validation

In [23]:
Adopt_Applicant_Info_count = pydb.read_sql_query("select count(*) as count from fcsq.Adopt_Applicant_Info")
Adopt_Applicant_Info_count

,count
0,255212


In [24]:
pydb.read_sql_query("select * from fcsq.adopt_applicant_info where case_number='HB12Z00228'")

,role,representative_role,role_model,party,case_number,person_given_first_name,person_family_name,company,postcode,gender,delete_flag
0,3232792,3437788,APLZ,2735994,HB12Z00228,Beverley Paula,Pierre,<NA>,CR2 9BA,2,N
1,3232791,3437788,APLZ,2736019,HB12Z00228,Donald Garfield,Finnikin And B Pierre,<NA>,CR2 9BA,1,N


## 10. adopt_respondent_info table - joins the roles, parties and address tables for respondents to get information on the respondents
<a name="adopt_respondent_info"></a>

### Drop the adopt_respondent_info table if it already exists and remove its data from the S3 bucket

In [25]:
drop_adopt_respondent_info = f"""
DROP TABLE IF EXISTS fcsq.adopt_respondent_info;
"""
pydb.start_query_execution_and_wait(drop_adopt_respondent_info)

# clean up previous adopt_respondent_info files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_respondent_info/").delete();

### Create the adopt_respondent_info table in Athena

In [26]:
create_adopt_respondent_info = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_respondent_info
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_respondent_info') AS
SELECT DISTINCT
  {database}.roles.ROLE, 
  {database}.roles.REPRESENTATIVE_ROLE, 
  {database}.roles.ROLE_MODEL, 
  {database}.roles.PARTY, 
  {database}.roles.CASE_NUMBER, 
  {database}.parties.GENDER, 
  {database}.addresses.POSTCODE, 
  {database}.roles.DELETE_FLAG
FROM 
  ({database}.roles INNER JOIN {database}.parties ON {database}.roles.PARTY = {database}.parties.PARTY) 
    LEFT JOIN {database}.addresses ON {database}.roles.ADDRESS = {database}.addresses.ADDRESS
WHERE 
    (((({database}.roles.ROLE_MODEL)='RSPA') AND (({database}.roles.DELETE_FLAG)='N')) 
    OR ((({database}.roles.ROLE_MODEL)='RSPZ') AND (({database}.roles.DELETE_FLAG)='N')))
    AND {database}.roles.mojap_snapshot_date = date '{snapshot_date}'
    AND {database}.parties.mojap_snapshot_date = date '{snapshot_date}'
    AND {database}.addresses.mojap_snapshot_date = date '{snapshot_date}';
"""

pydb.start_query_execution_and_wait(create_adopt_respondent_info);



#### adopt_respondent_info validation

In [27]:
adopt_respondent_info_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_respondent_info")
adopt_respondent_info_count

,count
0,301356


In [28]:
pydb.read_sql_query("select * from fcsq.adopt_respondent_info where case_number='HB12Z00228'")

,role,representative_role,role_model,party,case_number,gender,postcode,delete_flag
0,3232794,<NA>,RSPZ,2736020,HB12Z00228,2,CR7 8JF,N
1,3232774,3606633,RSPZ,2736056,HB12Z00228,1,CR7 8JF,N


## 11. applicants 3 table - takes the adopt_applicant_info table and reformats it, decoding gender and representative role values into strings
<a name="applicants_3"></a>

In [29]:
create_adopt_applicants_1 = f"""
SELECT T1.role,
    T1.representative_role,
    T1.role_model,
    T1.party,
    T1.case_number,
    T1.gender,
    case when cast(gender as varchar(1)) = '1' then 'Male'
    when cast(gender as varchar(1)) = '2' then 'Female'
    else 'Unknown' end as Gender_Decode

    FROM fcsq.adopt_applicant_info AS t1
    ORDER BY t1.Case_Number;
"""

pydb.create_temp_table(create_adopt_applicants_1,'adopt_applicants_1')



create_adopt_applicants_2 = f"""
SELECT DISTINCT 
    T1.case_number,
    T1.party,
    max(T1.representative_role) as Rep_Role,
    max(T1.gender_decode) as Gender_Max
    from __temp__.adopt_applicants_1 AS t1
    group by Case_number, party;
"""

pydb.create_temp_table(create_adopt_applicants_2,'adopt_applicants_2')


create_adopt_applicants_3= f"""
SELECT t1.case_number,
    t1.party as App_Party_ID,
    t1.Rep_Role,
    t1.Gender_Max,
    case when t1.Rep_Role IS NULL then 'N'
    when t1.Rep_Role IS NOT NULL then 'Y'
    End as REPRESENTATION,
    case when Rep_Role IS NULL AND Gender_Max = 'Female' then 'Unrep_Female'
    when Rep_Role IS NULL AND Gender_Max = 'Male' then 'Unrep_Male'
    when Rep_Role IS NULL AND Gender_Max = 'Unknown' then 'Unrep_Unknown'
    when Rep_Role IS NOT NULL AND Gender_Max = 'Female' then 'Rep_Female'
    when Rep_Role IS NOT NULL AND Gender_Max = 'Male' then 'Rep_Male'
    when Rep_Role IS NOT NULL AND Gender_Max = 'Unknown' then 'Rep_Unknown'
    else '' end as App_Rep_Cat
    
    from __temp__.adopt_applicants_2 AS t1;


"""

pydb.create_temp_table(create_adopt_applicants_3,'adopt_applicants_3')


## 12. Adopt_respondents temporary tables - takes the adopt_respondent_info table and reformats it, decoding gender and representative role values into strings
<a name="Adopt_respondents_temp"></a>

In [30]:
create_adopt_respondents_1 = f"""
SELECT T1.role,
    T1.representative_role,
    T1.role_model,
    T1.party,
    T1.case_number,
    T1.gender,
    case when cast(gender as varchar(1)) = '1' then 'Male'
    when cast(gender as varchar(1)) = '2' then 'Female'
    else 'Unknown' end as Gender_Decode

    FROM fcsq.adopt_respondent_info AS t1
    ORDER BY t1.Case_Number;
"""
#pydb.start_query_execution_and_wait(create_adopt_respondents_1)
pydb.create_temp_table(create_adopt_respondents_1,'adopt_respondents_1')



create_adopt_respondents_2 = f"""
    SELECT DISTINCT T1.case_number,
        T1.party,
        max(T1.representative_role) as Rep_Role,
        max(T1.gender_decode) as Gender_Max
    from __temp__.adopt_respondents_1 AS t1
    group by Case_number, party;
"""

pydb.create_temp_table(create_adopt_respondents_2,'adopt_respondents_2')
#pydb.start_query_execution_and_wait(create_adopt_respondents_2)


create_adopt_respondents_3= f"""
SELECT t1.case_number,
    t1.party as Resp_Party_ID,
    t1.Rep_Role,
    t1.Gender_Max,
    case when t1.Rep_Role IS NULL then 'N'
    when t1.Rep_Role IS NOT NULL then 'Y'
    End as REPRESENTATION,
    case when Rep_Role IS NULL AND Gender_Max = 'Female' then 'Unrep_Female'
    when Rep_Role IS NULL AND Gender_Max = 'Male' then 'Unrep_Male'
    when Rep_Role IS NULL AND Gender_Max = 'Unknown' then 'Unrep_Unknown'
    when Rep_Role IS NOT NULL AND Gender_Max = 'Female' then 'Rep_Female'
    when Rep_Role IS NOT NULL AND Gender_Max = 'Male' then 'Rep_Male'
    when Rep_Role IS NOT NULL AND Gender_Max = 'Unknown' then 'Rep_Unknown'
    else '' end as Resp_Rep_Cat
    
    from __temp__.adopt_respondents_2 AS t1;


"""
#pydb.start_query_execution_and_wait(create_adopt_respondents_3)
pydb.create_temp_table(create_adopt_respondents_3,'adopt_respondents_3')

## 13. Create adopt_app_rep table - joins the case data with the applicant representation data 
<a name="adopt_app_rep"></a>

In [31]:
adopt_app_rep_final = f"""
SELECT t1.YEAR, 
    t1.QUARTER,
    t1.CASE_NUMBER, 
    t1.Court,
    t2.App_Party_ID,
    t2.Representation,
    t2.Gender_Max as App_Gender,
    t2.App_Rep_Cat          
FROM __temp__.ADOPT_CASE_DATA_v3 t1
    LEFT JOIN __temp__.ADOPT_APPLICANTS_3 t2 ON (t1.CASE_NUMBER = t2.CASE_NUMBER);

"""

pydb.create_temp_table(adopt_app_rep_final,'adopt_app_rep_final')


In [32]:
adopt_app_rep_final_check = "SELECT COUNT(*) as Count from __temp__.adopt_app_rep_final"
pydb.read_sql_query(adopt_app_rep_final_check)

,count
0,109144


## 14. Create adopt_resp_rep table - joins the case data with the respondent representation data
<a name="Adopt_resp_rep"></a>

In [33]:
adopt_resp_rep_final = f"""
   SELECT t1.YEAR, 
        t1.QUARTER,
        t1.CASE_NUMBER, 
        t1.Court,
          t2.Resp_Party_ID,
          t2.Representation,
          t2.Gender_Max as Resp_Gender,
          t2.Resp_Rep_Cat
          
      FROM __temp__.ADOPT_CASE_DATA_v3 t1
           LEFT JOIN __temp__.ADOPT_RESPONDENTS_3 t2 ON (t1.CASE_NUMBER = t2.CASE_NUMBER);
"""

pydb.create_temp_table(adopt_resp_rep_final,'adopt_resp_rep_final')

In [34]:
adopt_resp_rep_final_check = "SELECT COUNT(*) as Count from __temp__.adopt_resp_rep_final"
pydb.read_sql_query(adopt_resp_rep_final_check)

,count
0,108493


## 15. Adopt_Hearing_Events table - takes the adoption hearing events from the hearings table and joins it with the events table data 
<a name="Adopt_Hearing_Events"></a>

### Drop the Adopt_Hearing_Events table if it already exists and remove its data from the S3 bucket

In [35]:
drop_Adopt_Hearing_Events = f"""
DROP TABLE IF EXISTS fcsq.Adopt_Hearing_Events;
"""
pydb.start_query_execution_and_wait(drop_Adopt_Hearing_Events)

# clean up previous Adopt_Hearing_Events files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_Hearing_Events/").delete();

### Create the Adopt_Hearing_Events table in Athena

In [36]:
create_Adopt_Hearing_Events = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_Hearing_Events
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_Hearing_Events') AS
SELECT {database}.hearings.EVENT,
  {database}.hearings.VACATED_FLAG,
  {database}.hearings.HEARING_TYPE,
  {database}.hearings.HEARING_DATE,
  {database}.events.RECEIPT_DATE,
  {database}.events.ERROR,
  {database}.events.CASE_NUMBER,
  {database}.events.EVENT_MODEL
FROM {database}.hearings
INNER JOIN {database}.events
ON {database}.hearings.EVENT            = {database}.events.EVENT
WHERE {database}.hearings.VACATED_FLAG IS NULL
AND {database}.events.ERROR             = 'N'
AND HEARING_DATE > date_parse('31-12-2009 00:00:00', '%d-%m-%Y %H:%i:%s')
AND (substring(case_number,5,1)='A' OR substring(case_number,5,1)='Z')
AND {database}.hearings.mojap_snapshot_date = date '{snapshot_date}' and {database}.events.mojap_snapshot_date = date '{snapshot_date}';
"""

pydb.start_query_execution_and_wait(create_Adopt_Hearing_Events);



#### Adopt_Hearing_Events validation

In [37]:
Adopt_Hearing_Events_count = pydb.read_sql_query("select count(*) as count from fcsq.Adopt_Hearing_Events")
Adopt_Hearing_Events_count

,count
0,542833


## 16. Adopt_Hearings_Cases table - takes the hearing events, filters by adoption event codes and adds a flag whether the hearing is the first in the case
<a name="Adopt_Hearings_Cases"></a>

### Drop the Adopt_Hearings_Cases table if it already exists and remove its data from the S3 bucket

In [38]:
drop_Adopt_Hearings_Cases = f"""
DROP TABLE IF EXISTS fcsq.Adopt_Hearings_Cases;
"""
pydb.start_query_execution_and_wait(drop_Adopt_Hearings_Cases)

# clean up previous Adopt_Hearings_Cases files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_Hearings_Cases/").delete();

### Create the Adopt_Hearings_Cases table in Athena

In [39]:
"""
Equivalent to Hearings_Adopt_V3
"""

create_Adopt_Hearings_Cases = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_Hearings_Cases
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_Hearings_Cases') AS
select t1.case_number,
    t1.error,
    t1.event,
    t1.event_model,
    t1.hearing_date,
    t1.hearing_type,
    t1.receipt_date,
    t1.vacated_flag,
    substring(Case_Number,5,1) AS Case_Type
    from fcsq.Adopt_Hearing_Events AS t1
    where t1.event_model in ('A8', 'A90', 'A91', 'G60')
    order by t1.case_number, t1.receipt_date;
"""

pydb.start_query_execution_and_wait(create_Adopt_Hearings_Cases)

create_Adopt_Hearings_Cases_v2 = f"""
SELECT *,
(case when row_number() over (partition by Case_Number order by receipt_date) = 1 then 1 else 0 end) as Case_Number_ID
FROM fcsq.Adopt_Hearings_Cases
"""

pydb.create_temp_table(create_Adopt_Hearings_Cases_v2,'adopt_hearings_cases_v2')



#### Adopt_Hearings_Cases validation

In [40]:
Adopt_Hearings_Cases_count = pydb.read_sql_query("select count(*) as count from __temp__.Adopt_Hearings_Cases_v2")
Adopt_Hearings_Cases_count

,count
0,345603


## 17. Hearing_Adopt_Applicants table - joins the cases from applicant representation table to the first hearing for the case in the adopt hearings cases table 
<a name="Hearing_Adopt_Applicants"></a>

### Drop the Hearing_Adopt_Applicants table if it already exists and remove its data from the S3 bucket

In [41]:
drop_Hearing_Adopt_Applicants = f"""
DROP TABLE IF EXISTS fcsq.Hearing_Adopt_Applicants;
"""
pydb.start_query_execution_and_wait(drop_Hearing_Adopt_Applicants)

# clean up previous Hearing_Adopt_Applicants files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Hearing_Adopt_Applicants/").delete();

### Create the Hearing_Adopt_Applicants table in Athena

In [42]:
create_Hearing_Adopt_Applicants = f"""
CREATE TABLE IF NOT EXISTS fcsq.Hearing_Adopt_Applicants
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Hearing_Adopt_Applicants') AS
SELECT t1.*,
t2.Case_Number_ID AS Hearing_Count
FROM __temp__.ADOPT_APP_REP_FINAL t1
LEFT JOIN __temp__.Adopt_Hearings_Cases_v2 t2 ON (t1.CASE_NUMBER = t2.CASE_NUMBER)
where t2.Case_Number_ID > 0;
"""

pydb.start_query_execution_and_wait(create_Hearing_Adopt_Applicants);



#### Hearing_Adopt_Applicants validation

In [43]:
Hearing_Adopt_Applicants_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_app_rep_final")
Hearing_Adopt_Applicants_count

,count
0,109144


## 18. Hearing_Adopt_Respondents table - joins the cases from respondents representation table to the first hearing for the case in the adopt hearings cases table 
<a name="Hearing_Adopt_Respondents"></a>

### Drop the Hearing_Adopt_Respondents table if it already exists and remove its data from the S3 bucket

In [44]:
drop_Hearing_Adopt_Respondents = f"""
DROP TABLE IF EXISTS fcsq.Hearing_Adopt_Respondents;
"""
pydb.start_query_execution_and_wait(drop_Hearing_Adopt_Respondents)

# clean up previous Hearing_Adopt_Respondents files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Hearing_Adopt_Respondents/").delete();

### Create the Hearing_Adopt_Respondents table in Athena

In [45]:
create_Hearing_Adopt_Respondents = f"""
CREATE TABLE IF NOT EXISTS fcsq.Hearing_Adopt_Respondents
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Hearing_Adopt_Respondents') AS
    SELECT t1.*,
    t2.Case_Number_ID AS Hearing_Count
    FROM __temp__.ADOPT_RESP_REP_FINAL t1
    LEFT JOIN __temp__.Adopt_Hearings_Cases_v2 t2 ON (t1.CASE_NUMBER = t2.CASE_NUMBER)
    where t2.Case_Number_ID > 0;
"""

pydb.start_query_execution_and_wait(create_Hearing_Adopt_Respondents);



#### Hearing_Adopt_Respondents validation

In [46]:
Hearing_Adopt_Respondents_count = pydb.read_sql_query("select count(*) as count from fcsq.Hearing_Adopt_Respondents")
Hearing_Adopt_Respondents_count

,count
0,101952


## 19. Adopt_App table - Groups the hearing_adopt_applicants table and produces a count per group
<a name="Adopt_App"></a>

### Drop the Adopt_App table if it already exists and remove its data from the S3 bucket

In [47]:
drop_Adopt_App = f"""
DROP TABLE IF EXISTS fcsq.Adopt_App;
"""
pydb.start_query_execution_and_wait(drop_Adopt_App)

# clean up previous Adopt_App files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_App/").delete();

### Create the Adopt_App table in Athena

In [48]:
create_Adopt_App = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_App
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_App') AS
SELECT
  'Adoption' AS CASE_TYPE,
  Year,
  Quarter,
  'Party' AS Category,
  'Applicant' AS PARTY,
   App_Gender AS Gender,
  Representation,
  Count (*) AS Count
FROM
  fcsq.HEARING_ADOPT_APPLICANTS
WHERE 
  Representation <> '' /*A very small number of cases from 2011/12 look into whether these should be recoded as N (gender is also blank)*/
GROUP BY
  'Adoption',
  Year,
  Quarter,
  'Party',
  'Applicant',
  App_Gender,
  Representation;
"""

pydb.start_query_execution_and_wait(create_Adopt_App);



#### Adopt_App validation

In [49]:
Adopt_App_count = pydb.read_sql_query("select count(*) as count from fcsq.Adopt_App")
Adopt_App_count

,count
0,240


## 20. Adopt_resp table - Groups the hearing_adopt_respondents table and produces a count per group
<a name="Adopt_resp"></a>

### Drop the Adopt_resp table if it already exists and remove its data from the S3 bucket

In [50]:
drop_Adopt_resp = f"""
DROP TABLE IF EXISTS fcsq.Adopt_resp;
"""
pydb.start_query_execution_and_wait(drop_Adopt_resp)

# clean up previous Adopt_resp files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_resp/").delete();

### Create the Adopt_resp table in Athena

In [51]:
create_Adopt_resp = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_resp
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_resp') AS
SELECT
  'Adoption' AS CASE_TYPE,
  Year,
  Quarter,
  'Party' AS Category,
  'Respondent' AS PARTY,
  Resp_Gender AS Gender,
  Representation,
  Count (*) AS Count
FROM
  fcsq.HEARING_ADOPT_RESPONDENTS
WHERE 
  Representation <> '' /*A very small number of cases from 2011/12 look into whether these should be recoded as N (gender is also blank)*/
GROUP BY
  'Adoption',
  Year,
  Quarter,
  'Party',
  'Applicant',
  Resp_Gender,
  Representation;
"""

pydb.start_query_execution_and_wait(create_Adopt_resp);



#### Adopt_resp validation

In [52]:
Adopt_resp_count = pydb.read_sql_query("select count(*) as count from fcsq.Adopt_resp")
Adopt_resp_count

,count
0,259


## 21. adopt_case table - groups and formats adopt_case_data_v3 table and gives a count for each group
<a name="adopt_case"></a>

### Drop the adopt_case table if it already exists and remove its data from the S3 bucket

In [53]:
drop_adopt_case = f"""
DROP TABLE IF EXISTS fcsq.adopt_case;
"""
pydb.start_query_execution_and_wait(drop_adopt_case)

# clean up previous adopt_case files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_case/").delete();

### Create the adopt_case table in Athena

In [54]:
create_adopt_case = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_case
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_case') AS
SELECT *,
    Count(*) as Count FROM
    (SELECT
        'Adoption' AS CASE_TYPE,
        Year,
        Quarter,
        'Cases' AS Category,
        'N/A' AS PARTY,
        'N/A' AS Gender,
        'N/A' AS Representation
    FROM
      __temp__.adopt_case_data_v3)
GROUP BY
  CASE_TYPE,
  Year,
  Quarter,
  Category,
  PARTY,
  Gender,
  Representation;
"""

pydb.start_query_execution_and_wait(create_adopt_case);



#### adopt_case validation

In [55]:
adopt_case_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_case")
adopt_case_count

,count
0,47


## 22. Adopt_Case_Hearings table - creates a count of all the cases with a hearing per quarter
<a name="Adopt_Case_Hearings"></a>

### Drop the Adopt_Case_Hearings table if it already exists and remove its data from the S3 bucket

In [56]:
drop_Adopt_Case_Hearings = f"""
DROP TABLE IF EXISTS fcsq.Adopt_Case_Hearings;
"""
pydb.start_query_execution_and_wait(drop_Adopt_Case_Hearings)

# clean up previous Adopt_Case_Hearings files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_Case_Hearings/").delete();

### Create the Adopt_Case_Hearings table in Athena

In [57]:
create_hearing_adopt_case =f"""
SELECT DISTINCT Year, Quarter, Case_Number
FROM fcsq.HEARING_ADOPT_Applicants;
"""

pydb.create_temp_table(create_hearing_adopt_case,'hearing_adopt_case')



create_Adopt_Case_Hearings = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_Case_Hearings
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_Case_Hearings') AS
SELECT *, Count(*) as Count FROM
    (SELECT
      'Adoption' AS CASE_TYPE,
      Year,
      Quarter,
      'Cases with a hearing' AS Category,
      'N/A' AS PARTY,
      'N/A' AS Gender,
      'N/A' AS Representation
    FROM
      __temp__.Hearing_ADOPT_Case)
GROUP BY
  CASE_TYPE,
  Year,
  Quarter,
  Category,
  PARTY,
  Gender,
  Representation;
"""

pydb.start_query_execution_and_wait(create_Adopt_Case_Hearings);



#### Adopt_Case_Hearings validation

In [58]:
Adopt_Case_Hearings_count = pydb.read_sql_query("select count(*) as count from fcsq.Adopt_Case_Hearings")
Adopt_Case_Hearings_count

,count
0,47


## 23. Adoption table - Joins the applicant/respondent representation count tables, case count table, and case hearing count tables 
<a name="Adoption"></a>

### Drop the Adoption table if it already exists and remove its data from the S3 bucket

In [59]:
drop_Adoption = f"""
DROP TABLE IF EXISTS fcsq.Adoption;
"""
pydb.start_query_execution_and_wait(drop_Adoption)

# clean up previous Adoption files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adoption/").delete();

### Create the Adoption table in Athena

In [60]:
create_Adoption = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adoption
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adoption') AS
SELECT
  *
FROM
 fcsq.ADOPT_APP
UNION ALL
SELECT
  *
FROM
  fcsq.ADOPT_RESP
UNION ALL
SELECT
  *
FROM
  fcsq.ADOPT_CASE
UNION ALL
SELECT
  *
FROM
  fcsq.ADOPT_CASE_HEARINGS;
"""

pydb.start_query_execution_and_wait(create_Adoption);



#### Adoption validation

In [61]:
Adoption_count = pydb.read_sql_query("select count(*) as count from fcsq.Adoption")
Adoption_count

,count
0,593


In [62]:
df = pydb.read_sql_query("SELECT * FROM fcsq.Adoption")
df

,case_type,year,quarter,category,party,gender,representation,count
0,Adoption,2017,3,Party,Applicant,Female,Y,41
1,Adoption,2015,3,Party,Applicant,Female,N,1226
2,Adoption,2016,2,Party,Applicant,Male,Y,44
3,Adoption,2013,1,Party,Applicant,Female,Y,59
4,Adoption,2022,3,Party,Applicant,Female,Y,3
...,...,...,...,...,...,...,...,...
588,Adoption,2015,3,Cases,N/A,N/A,N/A,1575
589,Adoption,2018,4,Cases,N/A,N/A,N/A,1301
590,Adoption,2019,4,Cases,N/A,N/A,N/A,1230
591,Adoption,2011,2,Cases,N/A,N/A,N/A,1152


In [63]:
legrep_lookup = f"""
SELECT
case_type || '|' || cast(year as varchar(10)) || '|' || cast(quarter as varchar(1)) || '|' as lookup,
case_type,
year,
cast(quarter as varchar(3)) as quarter,
category,
party,
gender,
representation,
count
from fcsq.adoption
order by lookup

"""
pydb.create_temp_table(legrep_lookup,'legrep_lookup')

pydb.read_sql_query("SELECT * from __temp__.legrep_lookup")

,lookup,case_type,year,quarter,category,party,gender,representation,count
0,Adoption|2011|1|,Adoption,2011,1,Cases with a hearing,N/A,N/A,N/A,1262
1,Adoption|2011|1|,Adoption,2011,1,Party,Applicant,Female,N,1014
2,Adoption|2011|1|,Adoption,2011,1,Party,Respondent,Male,N,841
3,Adoption|2011|1|,Adoption,2011,1,Party,Applicant,Male,N,1037
4,Adoption|2011|1|,Adoption,2011,1,Party,Respondent,Female,Y,63
...,...,...,...,...,...,...,...,...,...
588,Adoption|2022|3|,Adoption,2022,3,Party,Applicant,Male,Y,6
589,Adoption|2022|3|,Adoption,2022,3,Party,Respondent,Unknown,Y,7
590,Adoption|2022|3|,Adoption,2022,3,Cases,N/A,N/A,N/A,609
591,Adoption|2022|3|,Adoption,2022,3,Party,Respondent,Male,N,255


In [64]:
legrep_with_annual = f"""
SELECT
* from __temp__.legrep_lookup
UNION ALL
SELECT 
lookup,
case_type,
year,
quarter,
category,
party,
gender,
representation,
sum(count) from
(
SELECT
case_type || '|' || cast(year as varchar(10)) || '|' as lookup,
case_type,
year,
'N/A' as quarter,
category,
party,
gender,
representation,
count
from __temp__.legrep_lookup where year<>{latest_year}
)
group by 
lookup,
case_type,
year,
quarter,
category,
party,
gender,
representation
order by lookup
"""

pydb.create_temp_table(legrep_with_annual,'legrep_with_annual')

In [65]:
legrep_cases = f"""
SELECT lookup,
sum(count) as cases
from __temp__.legrep_with_annual
where category='Cases'
group by lookup
order by lookup
"""

pydb.create_temp_table(legrep_cases,'legrep_cases')

In [66]:
legrep_hearing_cases = f"""
SELECT lookup,
sum(count) as cases_hearing
from __temp__.legrep_with_annual
where category='Cases with a hearing'
group by lookup
order by lookup
"""
pydb.create_temp_table(legrep_hearing_cases,'legrep_hearing_cases')


In [67]:
legrep_app_rep = f"""
SELECT lookup,
sum(count) as app_rep
from __temp__.legrep_with_annual
where category='Party' and party = 'Applicant' and representation = 'Y'
group by lookup
order by lookup
"""

pydb.create_temp_table(legrep_app_rep,'legrep_app_rep')

In [68]:
legrep_app_unrep = f"""
SELECT lookup,
sum(count) as app_unrep
from __temp__.legrep_with_annual
where category='Party' and party = 'Applicant' and representation = 'N'
group by lookup
order by lookup
"""

pydb.create_temp_table(legrep_app_unrep,'legrep_app_unrep')

In [69]:
legrep_res_rep = f"""
SELECT lookup,
sum(count) as res_rep
from __temp__.legrep_with_annual
where category='Party' and party = 'Respondent' and representation = 'Y'
group by lookup
order by lookup
"""

pydb.create_temp_table(legrep_res_rep,'legrep_res_rep')

In [70]:
legrep_res_unrep = f"""
SELECT lookup,
sum(count) as res_unrep
from __temp__.legrep_with_annual
where category='Party' and party = 'Respondent' and representation = 'N'
group by lookup
order by lookup
"""

pydb.create_temp_table(legrep_res_unrep,'legrep_res_unrep')

In [71]:
legrep_total_parties = f"""
SELECT lookup,
sum(count) as total_parties
from __temp__.legrep_with_annual
where category='Party'
group by lookup
order by lookup
"""

pydb.create_temp_table(legrep_total_parties,'legrep_total_parties')

In [72]:
legrep_final_output = f"""
SELECT
t1.lookup,
t1.cases,
t2.cases_hearing,
t3.app_rep,
t4.app_unrep,
t5.res_rep,
t6.res_unrep,
t7.total_parties

from
((((((__temp__.legrep_cases t1 
INNER JOIN
__temp__.legrep_hearing_cases t2
ON 
t1.lookup = t2.lookup)

INNER JOIN 
__temp__.legrep_app_rep t3
ON 
t1.lookup = t3.lookup)

INNER JOIN 
__temp__.legrep_app_unrep t4
ON 
t1.lookup = t4.lookup)

INNER JOIN 
__temp__.legrep_res_rep t5
ON 
t1.lookup = t5.lookup)

INNER JOIN 
__temp__.legrep_res_unrep t6
ON 
t1.lookup = t6.lookup)

INNER JOIN 
__temp__.legrep_total_parties t7
ON 
t1.lookup = t7.lookup)
"""

df = pydb.read_sql_query(legrep_final_output)

df.to_csv(path_or_buf = 's3://alpha-family-data/CSVs/Adoption_legrep.csv',index=False)

## 24. Applicant_representation table - creates a table showing whether all, some, or none of the applicants for a case have representation
<a name="Applicant_representation"></a>

### Drop the Applicant_representation table if it already exists and remove its data from the S3 bucket

In [73]:
drop_Applicant_representation = f"""
DROP TABLE IF EXISTS fcsq.Applicant_representation;
"""
pydb.start_query_execution_and_wait(drop_Applicant_representation)

# clean up previous Applicant_representation files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Applicant_representation/").delete();

### Create the Applicant_representation table in Athena

In [74]:
create_applicants_1 = f"""
SELECT Distinct t1.Case_Number, t1.Party, MAX(t1.Representative_Role) as Max_Rep_Role
FROM fcsq.Adopt_Applicant_Info t1
Group by Case_Number, Party;
"""

pydb.create_temp_table(create_applicants_1,'applicants_1')

create_applicants_2 = f"""
SELECT  t1.*,
case when Max_Rep_Role IS NULL then 0
else 1
end as Rep_IND
FROM __temp__.Applicants_1 t1;
"""
pydb.create_temp_table(create_applicants_2,'applicants_2')

create_applicants_3 = f"""
SELECT Distinct t1.Case_Number,
Count(t1.Party) as CountOfParty,
SUM(t1.Rep_Ind) as SumOfRep_IND
FROM __temp__.Applicants_2 t1
Group by Case_Number;
"""
pydb.create_temp_table(create_applicants_3,'applicants_3')

create_Applicant_representation = f"""
CREATE TABLE IF NOT EXISTS fcsq.Applicant_representation
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Applicant_representation') AS
SELECT t1.Case_Number,
t1.CountOfParty,
t1.SumOfRep_IND,
CASE WHEN t1.SumOfRep_Ind > t1.CountOfParty then 'Error'
WHEN t1.SumOfRep_Ind = t1.CountOfParty then 'All'
WHEN t1.SumOfRep_Ind =0 then 'None' else 'Some'  end as App_Rep_Cat
FROM __temp__.Applicants_3 t1;
"""

pydb.start_query_execution_and_wait(create_Applicant_representation);



#### Applicant_representation validation

In [75]:
Applicant_representation_count = pydb.read_sql_query("select count(*) as count from fcsq.Applicant_representation")
Applicant_representation_count

,count
0,183969


## 25. Respondent_Representation table - creates a table showing whether all, some, or none of the respondents for a case have representation
<a name="Respondent_representation"></a>

### Drop the Respondent_Representation table if it already exists and remove its data from the S3 bucket

In [76]:
drop_Respondent_Representation = f"""
DROP TABLE IF EXISTS fcsq.Respondent_Representation;
"""
pydb.start_query_execution_and_wait(drop_Respondent_Representation)

# clean up previous Respondent_Representation files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Respondent_Representation/").delete();

### Create the Respondent_Representation table in Athena

In [77]:
create_respondents_1 = f"""
SELECT Distinct t1.Case_Number, t1.Party, MAX(t1.Representative_Role) as Max_Rep_Role
FROM fcsq.Adopt_Respondent_Info t1
Group by Case_Number, Party;
"""

pydb.create_temp_table(create_respondents_1,'respondents_1')

create_respondents_2 = f"""
SELECT  t1.*,
case when Max_Rep_Role IS NULL then 0
else 1
end as Rep_IND
FROM __temp__.respondents_1 t1;
"""
pydb.create_temp_table(create_respondents_2,'respondents_2')

create_respondents_3 = f"""
SELECT Distinct t1.Case_Number,
Count(t1.Party) as CountOfParty,
SUM(t1.Rep_Ind) as SumOfRep_IND
FROM __temp__.respondents_2 t1
Group by Case_Number;
"""
pydb.create_temp_table(create_respondents_3,'respondents_3')

create_Respondent_Representation = f"""
CREATE TABLE IF NOT EXISTS fcsq.Respondent_Representation
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Respondent_Representation') AS
SELECT t1.Case_Number,
t1.CountOfParty,
t1.SumOfRep_IND,
CASE WHEN t1.SumOfRep_Ind > t1.CountOfParty then 'Error'
WHEN t1.SumOfRep_Ind = t1.CountOfParty then 'All'
WHEN t1.SumOfRep_Ind =0 then 'None' else 'Some'  end as Res_Rep_Cat
FROM __temp__.Respondents_3 t1
"""

pydb.start_query_execution_and_wait(create_Respondent_Representation);



#### Respondent_Representation validation

In [78]:
Respondent_Representation_count = pydb.read_sql_query("select count(*) as count from fcsq.Respondent_Representation")
Respondent_Representation_count

,count
0,173255


## 26. Adopt_Disposals_Final table - joins the adopt_apps_and_orders_match table and the representation tables, joining timeliness and legal representation data for a case together
<a name="Adopt_Disposals_Final"></a>

In [79]:
create_ADOPT_APP_AND_ORDERS_WITH_REP = f"""
SELECT t1.*,
t2.APP_REP_CAT, 
t3.RES_REP_CAT
FROM fcsq.ADOPT_APPS_AND_ORDERS_MATCH AS t1
LEFT JOIN fcsq.Applicant_Representation AS t2 ON (t1.CASE_NUMBER = t2.CASE_NUMBER)
LEFT JOIN fcsq.RESPONDENT_REPRESENTATION as t3 ON t1.CASE_NUMBER = t3.CASE_NUMBER;
"""
pydb.create_temp_table(create_ADOPT_APP_AND_ORDERS_WITH_REP,'ADOPT_APP_AND_ORDERS_WITH_REP')

In [80]:
pydb.read_sql_query("SELECT Count(*) as COUNT from __temp__.adopt_app_and_orders_with_rep")

,count
0,84872


In [81]:
create_Adopt_Disposals_Final = f"""
SELECT t1.*,
t2.Region_Pre2014, 
t2.Region,
cast(t1.Year as varchar(3)) || '-Q' || cast(t1.quarter as varchar(3)) AS Quarter2, 
case when (t1.APP_REP_CAT Is Null Or t1.RES_REP_CAT Is Null)  then '5 Unknown'
    when t1.APP_REP_CAT='None' and t1.RES_REP_CAT ='None' then '4 Neither'
    when t1.APP_REP_CAT='None' and t1.RES_REP_CAT != 'None' then '3 Respondent Only'
    when t1.APP_REP_CAT != 'None' and t1.RES_REP_CAT = 'None' then '2 Applicant Only'
Else '1 Both'
End AS REP_CAT,
Case when t1.YEAR < 2014 then t2.Region_Pre2014
Else t2.Region
End As Final_Region

FROM __temp__.ADOPT_APP_AND_ORDERS_WITH_REP AS t1 LEFT JOIN fcsq.COURT_MV_FEB21_DFJ as t2
ON t1.DSP_COURT = cast(t2.Code as varchar(3));

"""

pydb.create_temp_table(create_Adopt_Disposals_Final,'Adopt_Disposals_Final')


create_Adopt_Disposals_Final_2 = f"""
SELECT *
FROM __temp__.ADOPT_DISPOSALS_FINAL
WHERE adoption = 'Adoption';
"""

pydb.create_temp_table(create_Adopt_Disposals_Final_2,'Adopt_Disposals_Final_2')



#### Adopt_Disposals_Final validation

In [82]:
Adopt_Disposals_Final_count = pydb.read_sql_query("select count(*) as count from __temp__.Adopt_Disposals_Final_2")
Adopt_Disposals_Final_count

,count
0,84872


## 27. Adopt_Quarterly table - groups the disposals_final table by quarter, providing a total and mean wait_weeks per quarter
<a name="Adopt_Quarterly"></a>

### Drop the Adopt_Quarterly table if it already exists and remove its data from the S3 bucket

In [83]:
drop_Adopt_Quarterly = f"""
DROP TABLE IF EXISTS fcsq.Adopt_Quarterly;
"""
pydb.start_query_execution_and_wait(drop_Adopt_Quarterly)

# clean up previous Adopt_Quarterly files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_Quarterly/").delete();

### Create the Adopt_Quarterly table in Athena

In [84]:
create_Adopt_Quarterly = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_Quarterly
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_Quarterly') AS
SELECT DISTINCT
        'Adoption' as type,
        year,
        'Q' || cast(quarter as varchar(3)) AS quarter,
        rep_cat,
        count(*) as n,
        avg(wait_weeks) as mean
    FROM 
        __temp__.ADOPT_DISPOSALS_FINAL_2
    WHERE year > 2010
    AND adoption='Adoption'
GROUP BY
    year,
    quarter,
    rep_cat
    
UNION ALL
SELECT DISTINCT
        'Adoption' as type,
        year,
        'Q' || cast(quarter as varchar(3)) AS quarter,
        'All' as rep_cat,
        count(*) as n,
        avg(wait_weeks) as mean
    FROM 
        __temp__.ADOPT_DISPOSALS_FINAL_2
    WHERE year > 2010 
    AND adoption='Adoption'
GROUP BY
    year,
    quarter
"""


pydb.start_query_execution_and_wait(create_Adopt_Quarterly)





{'QueryExecutionId': '31d085e0-2ff3-439d-9470-8d30578bea65',
 'Query': "CREATE TABLE IF NOT EXISTS fcsq.Adopt_Quarterly\nWITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_Quarterly') AS\nSELECT DISTINCT\n        'Adoption' as type,\n        year,\n        'Q' || cast(quarter as varchar(3)) AS quarter,\n        rep_cat,\n        count(*) as n,\n        avg(wait_weeks) as mean\n    FROM \n        mojap_de_temp_alpha_user_jdlv500.ADOPT_DISPOSALS_FINAL_2\n    WHERE year > 2010\n    AND adoption='Adoption'\nGROUP BY\n    year,\n    quarter,\n    rep_cat\n    \nUNION ALL\nSELECT DISTINCT\n        'Adoption' as type,\n        year,\n        'Q' || cast(quarter as varchar(3)) AS quarter,\n        'All' as rep_cat,\n        count(*) as n,\n        avg(wait_weeks) as mean\n    FROM \n        mojap_de_temp_alpha_user_jdlv500.ADOPT_DISPOSALS_FINAL_2\n    WHERE year > 2010 \n    AND adoption='Adoption'\nGROUP BY\n    year,\n    quarter",
 'Statemen

#### Adopt_Quarterly validation

In [85]:
Adopt_Quarterly_count = pydb.read_sql_query("select count(*) as count from fcsq.Adopt_Quarterly")
Adopt_Quarterly_count

,count
0,282


## 28. Adopt_Annual table - groups the disposals_final table annually, providing a total and mean wait_weeks annually
<a name="Adopt_Annual"></a>

### Drop the Adopt_Annual table if it already exists and remove its data from the S3 bucket

In [86]:
drop_Adopt_Annual = f"""
DROP TABLE IF EXISTS fcsq.Adopt_Annual;
"""
pydb.start_query_execution_and_wait(drop_Adopt_Annual)

# clean up previous Adopt_Annual files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/Adopt_Annual/").delete();

### Create the Adopt_Annual table in Athena

In [87]:
create_Adopt_Annual = f"""
CREATE TABLE IF NOT EXISTS fcsq.Adopt_Annual
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/Adopt_Annual') AS
SELECT DISTINCT
        'Adoption' as type,
        year,
        'N/A' as quarter,
        rep_cat,
        count(*) as n,
        avg(wait_weeks) as mean
    FROM 
        __temp__.ADOPT_DISPOSALS_FINAL_2
    WHERE year > 2010 
    AND year < {latest_year}
    AND adoption='Adoption'
GROUP BY
    year,
    rep_cat
    
UNION ALL
SELECT DISTINCT
        'Adoption' as type,
        year,
        'N/A' as quarter,
        'All' as rep_cat,
        count(*) as n,
        avg(wait_weeks) as mean
    FROM 
        __temp__.ADOPT_DISPOSALS_FINAL_2
    WHERE year > 2010 
    AND year < {latest_year}
    AND adoption='Adoption'
GROUP BY
    year
"""

pydb.start_query_execution_and_wait(create_Adopt_Annual);



#### Adopt_Annual validation

In [88]:
Adopt_Annual_count = pydb.read_sql_query("select count(*) as count from fcsq.Adopt_Annual")
Adopt_Annual_count



,count
0,66


## 29. adopt_timeliness_combined table - combines and orders the annual and quarterly timeliness data 
<a name="Adopt_timeliness_combined"></a>

### Drop the adopt_timeliness_combined table if it already exists and remove its data from the S3 bucket

In [89]:
drop_adopt_timeliness_combined = f"""
DROP TABLE IF EXISTS fcsq.adopt_timeliness_combined;
"""
pydb.start_query_execution_and_wait(drop_adopt_timeliness_combined)

# clean up previous adopt_timeliness_combined files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_timeliness_combined/").delete();

### Create the adopt_timeliness_combined table in Athena

In [90]:
create_adopt_timeliness_combined = f"""
CREATE TABLE IF NOT EXISTS fcsq.adopt_timeliness_combined
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Adoption/adopt_timeliness_combined') AS
SELECT * FROM fcsq.Adopt_Annual 
UNION ALL 
SELECT * FROM fcsq.Adopt_Quarterly ORDER BY type,year,quarter,rep_cat
"""

pydb.start_query_execution_and_wait(create_adopt_timeliness_combined);



#### adopt_timeliness_combined validation

In [91]:
adopt_timeliness_combined_count = pydb.read_sql_query("select count(*) as count from fcsq.adopt_timeliness_combined")
adopt_timeliness_combined_count

,count
0,348


In [92]:
pydb.read_sql_query("select * from fcsq.adopt_timeliness_combined")

,type,year,quarter,rep_cat,n,mean
0,Adoption,2011,N/A,1 Both,95,28.969925
1,Adoption,2011,N/A,2 Applicant Only,252,19.257370
2,Adoption,2011,N/A,3 Respondent Only,179,19.018356
3,Adoption,2011,N/A,4 Neither,3968,15.529558
4,Adoption,2011,N/A,5 Unknown,172,15.033223
...,...,...,...,...,...,...
343,Adoption,2022,Q3,2 Applicant Only,7,20.224490
344,Adoption,2022,Q3,3 Respondent Only,25,34.222857
345,Adoption,2022,Q3,4 Neither,558,22.130312
346,Adoption,2022,Q3,5 Unknown,8,19.553571


In [93]:
lookup_table = f"""
SELECT 
CASE when quarter = 'N/A' THEN 
type || '|' ||cast(year as varchar(20))||'|'
ELSE
type || '|' ||cast(year as varchar(20))||'|' || quarter || '|' end as lookup,
rep_cat,
n,
mean
from fcsq.adopt_timeliness_combined;
"""

pydb.create_temp_table(lookup_table, 'lookup_table')

In [94]:
df = pydb.read_sql_query("SELECT CASE when quarter = 'N/A' THEN type || '|' ||cast(year as varchar(20))||'|' ELSE type || '|' ||cast(year as varchar(20))||'|' || quarter || '|' end as lookup, rep_cat, n, mean from fcsq.adopt_timeliness_combined")
df.head()

,lookup,rep_cat,n,mean
0,Adoption|2011|,1 Both,95,28.969925
1,Adoption|2011|,2 Applicant Only,252,19.257370
2,Adoption|2011|,3 Respondent Only,179,19.018356
3,Adoption|2011|,4 Neither,3968,15.529558
4,Adoption|2011|,5 Unknown,172,15.033223


In [95]:
n_count_table = f"""
SELECT lookup, rep_cat, n
FROM __temp__.lookup_table 
ORDER BY lookup;
"""

pydb.create_temp_table(n_count_table, 'n_count_table')

In [96]:
df = pydb.read_sql_query("SELECT * FROM __temp__.n_count_table order by lookup;")
df.head()

,lookup,rep_cat,n
0,Adoption|2011|,1 Both,95
1,Adoption|2011|,2 Applicant Only,252
2,Adoption|2011|,3 Respondent Only,179
3,Adoption|2011|,4 Neither,3968
4,Adoption|2011|,5 Unknown,172


In [97]:
n_count_table_final = f"""
SELECT
lookup,
map_n['1 Both'] as "1_Bothn",
map_n['2 Applicant Only'] as "2_Applicant_Onlyn",
map_n['3 Respondent Only'] as "3_Respondent_Onlyn",
map_n['4 Neither'] as "4_Neithern",
map_n['5 Unknown'] as "5_Unknown",
map_n['All'] as "Alln"
from (
SELECT 
lookup,
map_agg(
rep_cat,
n
) map_n
FROM __temp__.n_count_table
group by lookup) 
order by lookup;
"""

pydb.create_temp_table(n_count_table_final,'n_count_table_final')

In [98]:
df = pydb.read_sql_query("SELECT * FROM __temp__.n_count_table_final order by lookup;")
df.head()

,lookup,1_bothn,2_applicant_onlyn,3_respondent_onlyn,4_neithern,5_unknown,alln
0,Adoption|2011|,95,252,179,3968,172,4666
1,Adoption|2011|Q1|,19,63,39,909,46,1076
2,Adoption|2011|Q2|,18,52,44,1050,38,1202
3,Adoption|2011|Q3|,27,63,44,968,58,1160
4,Adoption|2011|Q4|,31,74,52,1041,30,1228


In [99]:
mean_table = f"""
SELECT lookup, rep_cat, mean
FROM __temp__.lookup_table 
ORDER BY lookup;
"""

pydb.create_temp_table(mean_table, 'mean_table')

In [100]:
df = pydb.read_sql_query("SELECT * FROM __temp__.mean_table order by lookup;")
df.head()

,lookup,rep_cat,mean
0,Adoption|2011|,1 Both,28.969925
1,Adoption|2011|,2 Applicant Only,19.257370
2,Adoption|2011|,3 Respondent Only,19.018356
3,Adoption|2011|,4 Neither,15.529558
4,Adoption|2011|,5 Unknown,15.033223


In [101]:
mean_table_final = f"""
SELECT
lookup,
map_mean['1 Both'] as "1_Both_mean",
map_mean['2 Applicant Only'] as "2_Applicant_Only_mean",
map_mean['3 Respondent Only'] as "3_Respondent_Only_mean",
map_mean['4 Neither'] as "4_Neither_mean",
map_mean['5 Unknown'] as "5_Unknown_mean",
map_mean['All'] as "All_mean"
from (
SELECT 
lookup,
map_agg(
rep_cat,
mean
) map_mean
FROM __temp__.mean_table
group by lookup) 
order by lookup;
"""

pydb.create_temp_table(mean_table_final,'mean_table_final')

In [102]:
csv_output_table ="""
SELECT 
t1.lookup,
t1."1_Bothn",
t2."1_Both_mean",
t1."2_Applicant_Onlyn",
t2."2_Applicant_Only_mean",
t1."3_Respondent_Onlyn",
t2."3_Respondent_Only_mean",
t1."4_Neithern",
t2."4_Neither_mean",
t1."5_Unknown",
t2."5_Unknown_mean",
t1."Alln",
t2."All_mean"
from 
__temp__.n_count_table_final t1
INNER JOIN
__temp__.mean_table_final t2
on (t1.lookup = t2.lookup);
"""


df = pydb.read_sql_query(csv_output_table)
df.to_csv(path_or_buf = 's3://alpha-family-data/CSVs/Adoption_Timeliness.csv',index=False)

In [103]:
view_table = pydb.read_sql_query("SELECT * FROM fcsq.adopt_timeliness_combined")
view_table.pivot_table(index=['type','year','quarter'],columns=['rep_cat'],values = ['n','mean'],aggfunc=sum, fill_value=0).swaplevel(axis=1).sort_index(axis=1)


rep_cat               1 Both                2 Applicant Only       3 Respondent Only      4 Neither               5 Unknown              All                    
                                   mean   n              mean    n              mean    n              mean     n              mean    n              mean     n
type     year quarter                                                                                                                                           
Adoption 2011 N/A             28.969925  95         19.257370  252         19.018356  179         15.529558  3968         15.033223  172         16.120078  4666
              Q1              32.278195  19         21.970522   63         20.758242   39         15.853214   909         14.145963   46         16.606213  1076
              Q2              32.619048  18         19.384615   52         16.620130   44         14.730476  1050         15.293233   38         15.286665  1202
              Q3              26.396825  27         16.682540   63         19.961039   44         15.817739   968         14.298030   58         16.192118  1160
              Q4              27.064516  31         19.050193   74         18.945055   52         15.784960  1041         17.485714   30         16.441833  1228
         2012 N/A             29.933333  75         23.284467  229         19.541591  158         14.429370  4654         18.880272  105         15.284702  5221
              Q1              27.296703  13         22.721550   59         20.084249   39         14.801991  1062         17.520000   25         15.556284  1198
              Q2              30.123810  15         26.263039   63         17.937729   39         14.487096  1096         20.220408   35         15.538118  1248
              Q3              33.366460  23         21.293956   52         21.833977   37         14.630694  1198         19.529762   24         15.501392  1334
              Q4              27.952381  24         22.358442   55         18.531561   43         13.889941  1298         17.523810   21         14.638842  1441
         2013 N/A             28.203810  75         20.808379  208         19.068161  153         14.196296  5531         15.414747   62         14.734829  6029
              Q1              37.376623  22         21.320106   54         21.282738   48         14.628715  1192         19.129870   11         15.556142  1327
              Q2              23.292208  22         18.232143   56         17.894180   54         14.060805  1459         14.731429   25         14.469501  1616
              Q3              29.659341  13         19.273608   59         18.464286   28         13.576372  1445         17.704762   15         14.053297  1560
              Q4              21.944444  18         26.120879   39         17.937888   23         14.599104  1435         10.129870   11         14.998315  1526
         2014 N/A             30.596059  58         22.309623  239         23.279971  199         15.270509  6135         31.181406   63         16.042469  6694
              Q1              20.285714   5         18.060440   52         22.109023   38         14.616213  1512         14.983193   17         14.923118  1624
              Q2              35.344538  17         24.778061   56         24.167702   46         14.656734  1539         12.047619   15         15.443856  1673
              Q3              28.849206  18         21.024554   64         22.827068   57         16.594971  1602         82.704762   15         17.649040  1756
              Q4              30.722222  18         24.771855   67         23.788177   58         15.143725  1482         18.026786   16         16.041351  1641
         2015 N/A             29.085253  62         20.489993  207         24.968254  225         15.268814  5579         15.863265   70         15.946234  6143
              Q1              29.061224  14         21.018433   62         24.060268   64         15.084366  1441         12.957143   20